### Q1  有10项 其中的申请材料部分 部分url有弹窗 无法获取url
### Q2 BasicDeal get_filepath 可以改进 如果len(url) == 0

In [29]:
from selenium import webdriver
import re

import time

from bs4 import BeautifulSoup

from util import *
from RightPage import Right

import json

In [30]:
# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11445300007175700L3440109007001'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(2)
# text = browser.page_source

In [31]:
class BasicPage:
    '''  基类 用于disp/fill函数处理'''
    def __init__(self):
#         self.f = fout
        self.content = {}
    
    def fill_content(self,tc1,tc2):
        assert len(tc1) == len(tc2)
        for k,v in zip(tc1,tc2):
            if len(v) == 0:
                continue
            else:
                self.content[k] = v
    
    def disp(self,fout):
        tmp = json.dumps(self.content,ensure_ascii=False,indent = 4)
        fout.write(tmp+'\n')


In [32]:
class BasicInfo(BasicPage):
    def __init__(self,text):
        super().__init__()
        self.binfo = {} ### 基本信息
        self.approval = {} ### 审批信息
        self.result = {} ### 审批结果
        self.code = {} ## 编码代码
        self.special_pro = {} ## 特殊程序
        self.others = {} ## 其他信息
        pat = '<h2 id="matters-part1".*?>基本信息</h2>(.*?)收起'
        s1 = re.search(pat,text,re.S)
        if s1 == None:
            self.text = None
        else:
            s1 = s1.groups()[0]
            self.text = s1
        
        
    def get_binfo(self):
        '''
        获得页面 起始表格 的信息
        '''
        text = self.text
        pat = '<div class="table-collapse-wrapper collapsed">(.*?)</tbody>'
        info_part = re.search(pat,text,re.S)   ### 得到基本信息部分的网页源代码，然后在源代码中找到相应数据
        if info_part == None:
            print('Info Part None ')
            return
        info_part = info_part.groups()[0] 
#         print(info_part)
        
#         pat1 = r'<th>(.*?)</th>.*?<td.*?><p class="matters-truncate">(.*?)</p>'
        pat1 = r'<th>(.*?)</th>.*?<td.*?>(.*?)</td>'
        tb = re.findall(pat1,info_part,re.S)  ### 得到具体信息

        for item in tb:
            k,v = item
            if k == '法定办结时限' or k == '承诺办结时限':
                btm = BeautifulSoup(self.text)
                v = handle_time(v,btm)
            if type(v) == str:
                v = replace_brac(v)
                v = replace_text(v)
            self.binfo[k] = v
    
    def get_approval(self):
        ''' 获得审批信息 '''
        text = self.text
        pat = r'<h3>审批信息</h3>(.*?)</tbody>'
        raw_html = re.search(pat,text,re.S) ## 获得审批结果部分的源代码
        if raw_html == None:
#             print('Approval None ')
            return
        raw_html = raw_html.groups()[0]
#         print(raw_html)
        pat1 = r'<th>(.*?)</th>.*?<td>(.*?)</td>'
        tb = re.findall(pat1,raw_html,re.S)
        for k,v in tb:
            v = replace_text(v)
            v = replace_brac(v)
            self.approval[k] = v.replace('\n','').replace('\t','')
    
    
    def get_result(self):
        ''' 获得审批结果'''
        text = self.text
        pat = r'<h3>审批结果</h3>(.*?)</tbody>'
#         pat = r'<table class="matters-table matters-table-material matters-table-info".*?>(.*?)</table>'
        raw_html = re.search(pat,text,re.S) ## 获得审批结果部分的源代码
        if raw_html == None:
            print('Result None ')
            return
        raw_html = raw_html.groups()[0]
        
        pat1 = r'<th><div style="min-width:.*?">(.*?)</div></th>'
        head = re.findall(pat1,raw_html,re.S)
        
        pat2 = r'<td>(.*?)</td>'
        body = re.findall(pat2,raw_html,re.S)
        
        for k,v in zip(head,body):
            v = replace_text(v)
            v = replace_brac(v)
            self.result[k] = v
            
    def get_code(self):
        '''获得编码代码'''
        text = self.text
        pat = r'<h3>编码代码</h3>.*?<table.*?>(.*?)</table>'
        raw_html = re.search(pat,text,re.S)  ## 获得编码代码的网页源码
#         print(raw_html)
        if raw_html == None:
            print('Code None')
            return
        raw_html = raw_html.groups()[0]
#         print(raw_html)
        
        pat1 = r'<th>(.*?)</th>.*?<td>(.*?)</td>'
        tb = re.findall(pat1,raw_html,re.S)
        for k,v in tb:
            v = re.sub(r'<.*?>', "", v)  ### 去除实施主体编码的头尾源码<p .*?></p>
            self.code[k] = v.replace('\n','').replace('\t','')
            
    def get_special_pro(self):
        ''' 处理特殊程序这个表格 '''
        text = self.text
        pat = r'<h3>特殊程序</h3>.*?<table.*?>(.*?)</table>'
        raw_html = re.search(pat,text,re.S)  ## 获得编码代码的网页源码
        if raw_html == None:
            print('Procedure None')
            return
        raw_html = raw_html.groups()[0]
#         print(raw_html)
        
        pat1 = '<th>.*?([\u4e00-\u9fa9]+).*?</th>'
        th = re.findall(pat1,raw_html,re.S)
        
        pat1 = '<td>(.*?)</td>'
        td = re.findall(pat1,raw_html,re.S)
        
        for k,v in zip(th,td):
            v = replace_brac(v)
            v = replace_text(v)
            self.special_pro[k] = v
            
    def get_others(self):
        ''' 处理其他信息这个表格'''
        text = self.text
        pat = r'<h3>其他信息</h3>.*?<table.*?>(.*?)</table>'
        raw_html = re.search(pat,text,re.S)  ## 获得编码代码的网页源码
        if raw_html == None:
            print('Procedure None')
            return
        raw_html = raw_html.groups()[0]
#         print(raw_html)
        pat1 = '<th>.*?([\u4e00-\u9fa9]+).*?</th>'
        th = re.findall(pat1,raw_html,re.S)
        
        pat1 = '<td>(.*?)</td>'
        td = re.findall(pat1,raw_html,re.S)
        
        for k,v in zip(th,td):
            v = replace_brac(v)
            v = replace_text(v)
            self.others[k] = v
            
    def handle_infos(self):
        self.get_binfo()
        self.get_approval()
        self.get_result()
        self.get_code()
        self.get_special_pro()
        self.get_others()
        
        tc1 = ['基础信息','审批信息','审批结果','编码代码','特殊程序','其他信息']
        tc2 = [self.binfo,self.approval,self.result,self.code,self.special_pro,self.others]
#         self.content['基础信息'] = [self.binfo]
#         self.content['审批结果'] = [self.result]
#         self.content['编码代码'] = [self.code]
#         self.content['特殊程序'] = [self.special_pro]
#         self.content['其他信息'] = [self.others]
        
        self.fill_content(tc1,tc2)
        

# 测试该类
# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300MB2C94128A3440217281000'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(0.5)
# text = browser.page_source

# bi = BasicInfo(text)
# bi.handle_infos()
# print("***起始表格***")
# print(bi.binfo)
# print('***审批信息***')
# print(bi.approval)
# print('***审批结果***')
# print(bi.result)
# print('***编码代码***')
# print(bi.code)
# print('***特殊程序***')
# print(bi.special_pro)
# print('***其他信息***')
# print(bi.others)

# '''
# OUTPUT:
# ***起始表格***
# {'事项名称': '普通高校入学考试三侨生证明办理', '事项类型': '行政确认', '事项名称短语': '三侨生证明', '日常用语': '三侨生', '法定办结时限': '30 (工作日)查看说明', '承诺办结时限': '15 (工作日)查看说明', '实施主体': '法定机关', '办件类型': '承诺件', '办理形式': '窗口办理,网上办理', '到办事现场次数': '1', '数量限制': '无', '必须现场办理原因': '申请人提供的申请材料中包含国外证明材料等原件。', '是否收费': '否', '是否支持预约办理': '否', '是否支持自助终端办理': '否', '在线预约地址': '否', '是否进驻政务大厅': '是', '是否支持网上支付': '否', '委托部门': '无', '是否告知承诺制': '否'}
# ***审批信息***
# {'行使层级': '市级', '权力来源': '法定本级行使', '审批服务形式': '网上办', '业务系统': '深圳市统一申办受理平台', '通办范围': '无', '联办机构': '无', '事项状态': '使用中', '事项版本': '10'}
# ***审批结果***
# {'序号': '1', '名称': '广东省“三侨生”证明书', '类型': '其他', '模板': '三桥生.jpg(http://www.gdzwfw.gov.cn//static.gdzwfw.gov.cn/guide/files/2dca27a0-bfb0-4b6d-9dd1-e4bd42d2ef0f)', '样例': '三侨生证明书样本.jpg(http://www.gdzwfw.gov.cn//static.gdzwfw.gov.cn/guide/files/2146b77e-320d-4556-883a-39ea99785168)', '关联状态': '已关联电子证照'}
# ***编码代码***
# {'基本编码': '000744002000', '实施主体编码': 'TE4403000075466630', '实施编码': 'TE44030000754666303000744002000', '业务办理项编码': '无'}
# ***特殊程序***
# {'类型': '无', '总时限': '无', '总时限说明': '无', '特殊程序说明': '无'}
# ***其他信息***
# {'乡镇街道名称': '无', '乡镇街道代码': '无', '村镇社区名称': '无', '村镇社区代码': '无', '移动端是否对接单点登录': '否', '移动端办理地址': '无', '计算机端是否对接单点登录': '是', '计算机端在线办理地址': '点击访问(http://wsbs.sz.gov.cn/apply/ui/4403000000000075466631000303905001)', '计划生效日期': '无', '计划取消日期': '无'}

# '''

In [33]:
class BasicAccept(BasicPage):
    
    ''' 处理受理标准 '''
    def __init__(self,text):
        super().__init__()
        self.scope = {}  ## 受理范围
        self.condition = {} ## 受理条件
        
        pat1 = r'<h3>受理范围</h3>(.*?)</tbody>'
        self.text1 = S_search(pat1,text)
        
        pat2 = r'<h3>受理条件</h3>(.*?)</div>'
        self.text2 = S_search(pat2,text)
        
        
    def get_condition(self):
        pat = '<p.*?>(.*?)</p>'
        tmp = S_search(pat,self.text2)
        self.condition['受理条件'] = tmp
        
    def get_scope(self):
        pat1 = r'<th.*?>(.*?)</th>.*?<td.*?>(.*?)</td>'
        tb = re.findall(pat1,self.text1,re.S)
        for k,v in tb:
            v = replace_text(v)
            v = replace_brac(v)
            self.scope[k] = v
            
    def handle_acc(self):
        
        if self.text1 == '':
            return
        
        self.get_condition()
        self.get_scope()
        
        tc1 = ['受理范围','受理条件']
        tc2 = [self.scope,self.condition]
        self.fill_content(tc1,tc2)
    
            

In [34]:
def handle_evi(v,btm):
    '''
    input: 材料中的详细依据
    out: 格式化后的信息  [ 法规明晨，文号，内容等 ]
    '''
    res = []
    text1 = v
    if 'javascrip' in text1 and 'data-target' in text1:
        patj = 'data-target="(.*?)"'
        idsx = S_search(patj,text1)
        if idsx == "":
            return ""
        else:
            
            idsx = idsx.replace('#','')
            text2 = btm.find_all(id = idsx)
            
            if len(text2) != 0:
                text2 = str(text2[0])
            else:
                text2 = ''
#             print(text2)
            patbd = r'<table.*?>(.*?)</table>'
#             patbd = '<div class="modal-bd">.*?<p>(.*?)</p>' ###############################################################毁在这个<p></p>
            text3 = S_search(patbd,text2)
            
            ths = re.findall('<th>(.*?)</th>',text3,re.S)
            tds = re.findall('<td>(.*?)</td>',text3,re.S)
            
            edict = {}
            for k,v in zip(ths,tds):
                k = replace_brac(k)
                k = replace_text(k)
                v = replace_brac(v)
                v = replace_text(v)
                edict[k] = v
#             print(edict)
            return str(edict)
                
    else:
        return ''
           
    
def handle_req(v,btm):
    '''
    input: 材料中的其他要求
    out: 格式化后的信息  [ 法规明晨，文号，内容等 ]
    '''
    res = {}
    text1 = v
    if 'javascrip' in text1 and 'data-target' in text1:
        patj = 'data-target="(.*?)"'
        idsx = S_search(patj,text1)
        if idsx == "":
            return ""
        else:
            
            idsx = idsx.replace('#','')
            text2 = btm.find_all(id = idsx)
            
            if len(text2) != 0:
                text2 = str(text2[0])
            else:
                text2 = ''
#             print(text2)
#             patbd = r'<table.*?>(.*?)</table>'
            patbd = '<div class="modal-bd">.*?<p>(.*?)</p>' 
            text3 = S_search(patbd,text2)
            text3 = text3.replace('\n','').replace('\t','')
            tlst = text3.split('<br/>')
            if len(tlst) == 1:
                return text3
            else:
                for tt in tlst[:-1]:
                    tt = tt.split('：')
                    if len(tt) < 2:
                        res['其他'] = tt[0]
                    else:
#                     print(tt)
                        res[tt[0]]= tt[1]
                    
            return str(res)
                
    else:
        return ''

def handle_source(v,btm):
    '''
    input: 材料中的填报须知与来源
    out: 格式化后的信息  [ 法规明晨，文号，内容等 ]
    '''
    res = ''
    text1 = v
    if 'javascrip' in text1 and 'data-target' in text1:
        patj = 'data-target="(.*?)".*?填报须知'
        idsx = S_search(patj,text1)
        if idsx == "":
            return ""
        else:
            
            idsx = idsx.replace('#','')
            text2 = btm.find_all(id = idsx)
            
            if len(text2) != 0:
                text2 = str(text2[0])
            else:
                text2 = ''
#             print(text2)
#             patbd = r'<table.*?>(.*?)</table>'
            patbd = '<div class="modal-bd">.*?<p>(.*?)</p>' 
            text3 = S_search(patbd,text2)
            text3 = text3.replace('\n','').replace('\t','')
            res = '填报须知'+'('+text3+')'
#             print(res)
            
        patj = 'data-target="(.*?)".*?来源渠道'
        tmp = re.search(patj,text1)
        if tmp == None:
            idsx = ''
        else:
            idsx = tmp.groups()[0]
#         idsx = S_search(patj,text1)
        if idsx == "":
            return ""
        else:
            
            idsx = idsx.replace('#','')
            text2 = btm.find_all(id = idsx)
            
            if len(text2) != 0:
                text2 = str(text2[0])
            else:
                text2 = ''
#             print(text2)
#             patbd = r'<table.*?>(.*?)</table>'
            patbd = '<div class="modal-bd">(.*?)</div>' 
            text3 = S_search(patbd,text2)
            text3 = text3.replace('\n','').replace('\t','')
            s2 = re.findall('<p>(.*?)</p>',text3,re.S)
            tdict = {}
            for s1 in s2:
                s1 = s1.split('：')
                tdict[s1[0]] = s1[1]
        
            if res == "":
                cflag = ''
            else:
                cflag = ','
            
            
            res = res + cflag + '来源渠道'+'('+str(tdict)+')'
            return res
                
    else:
        return ''
    

### 前面均为本类所需要的函数
class Materials(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        soup = BeautifulSoup(text,'lxml')#选择lxml作为解析器
        tb1 = soup.find_all(id = 'matters-cond')
        if len(tb1) == 0:
            tb1 = ''
        else:
            tb1 = str(tb1[0])
        self.soup = soup
        pat1 = '<thead>(.*?)</thead>'
        self.thead = S_search(pat1,tb1)
        pat2 = '<tbody.*?>(.*?)</tbody>'
        self.tbody = S_search(pat2,tb1)
        
        self.appli = []
        
    def handle_td(self,text):
        ''' 处理材料表格 '''
#         print(text)
#         print('*****')
        if '查看材料依据' in text:  ## 处理材料依据
            evi = handle_evi(text,self.soup)
        else:
            evi = ''
#         print(evi+'*********')
#             return evi
        if '其他要求' in text:
            req  = handle_req(text,self.soup)
        else:
            req = ''
            
        if '来源渠道' in text or '填报须知' in text:
            source = handle_source(text,self.soup)
            return source
        
        ''' 整理表格内容，融合url '''
        if "FILEPATH" not in text:        
            t = replace_brac(text)
            t = replace_text(t)
            t = t.replace('<br>',',').replace('<br/>',',')
            if req == '' and evi == '':
                return t
            elif req != '':
                return t+'('+req+')'
            elif evi != '':
                return t+'('+evi+')'
            
        else:
            tstr = ''
            tlst = re.findall('<a.*?"FILEPATH":"(.*?)".*?>(.*?)<.*?</a>',text,re.S)
            for t in tlst:
                a,b = t
                a = 'http:'+a
                if tstr == '':
                    fstr = ''
                else:
                    fstr = ','
                tstr = tstr + fstr + b + '('+ a + ')'
            return tstr
        
    def get_appli(self):
        hpat = '<th>.*?<div.*?>(.*?)</div>.*?</th>'
        head = re.findall(hpat,self.thead,re.S)  ## 表头
        
        hpat = '<td.*?>(.*?)</td>'
        body1 = re.findall(hpat,self.tbody,re.S)
        body = []
#         print(body1)
        
        i = 0
        for b in body1:
            
#             b1 = replace_text(b)
            b1 = self.handle_td(b)
            body.append(b1)
            i += 1
        
        dt = {}
        for i in range(len(body)):
            dt[head[i%6]] = body[i]
            if i%6 == 5:
                self.appli.append(dt)
                dt = {}
                
    def handle_appli(self):
        if self.thead == '':
            return
        self.get_appli()
        
        tc1 = ['申请材料']
        tc2 = [self.appli]
        self.fill_content(tc1,tc2)


In [35]:
class BasicWin(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        soup = BeautifulSoup(text,'lxml')
        tmp = soup.find_all(class_='matters-window')
        if len(tmp) == 0:
            self.text = ''
        else:
            self.text = str(tmp[0])
         
        self.win = {}
        
    def get_win(self):
        pat = r'<p.*?>(.*?)</p>'
        con = re.findall(pat,self.text,re.S)
        title = con[0]
        self.win['title'] = title
        for c in con[1:]:
            if len(c.split('：',1)) == 1:
                continue
            a,b = c.split('：',1)
            a = replace_text(a)
            b = replace_text(b)
#             print(a,b)
            self.win[a] = b
    def handle_win(self):
        if self.text == '':
            return
        self.get_win()
        self.content = self.win

In [36]:
class BasicFee(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        pat = '<h2 id="matters-part7".*?>收费项目信息</h2>(.*?)</table>'
        self.text = S_search(pat,text)
        
        self.fee = {}
#         self.text = replace_text(self.text)
#         self.text = replace_brac(self.text)
    
    def get_fee(self):
        thead = re.findall('<th>(.*?)</th>',self.text,re.S)
        tbody = re.findall('<td>(.*?)</td>',self.text,re.S)
#         print(thead)
#         print(tbody)
        for k,v in zip(thead,tbody):
            k = replace_brac(k)
            k = replace_text(k)
            
            v = replace_brac(v)
            v = replace_text(v)
            
#             print(k,v)
            
            self.fee[k] = v
            
    
    def handle_fee(self):
        if self.text == '' or '收费项目名称' not in self.text:
#             print('无收费项目')
            return 
        self.get_fee()
        tc1 = ['收费项目']
        tc2 = [self.fee]
        self.fill_content(tc1,tc2)

# bf = BasicFee(text)

# bf.handle_fee()

# bf.content

In [37]:
class BasicAgent(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        pat = r'<h2 id="matters-part8".*?>中介服务</h2>(.*?)</div>'
        self.text = S_search(pat,text)
        self.text = replace_text(self.text)
        self.text = replace_brac(self.text)
        
    def handle_agent(self):
        if self.text == '':
            return
        tc1 = ['中介']
        tc2 = [self.text]
        self.fill_content(tc1,tc2)  

In [38]:
class BasicSup(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        self.consult = {} ### 咨询方式
        self.complaint = {} ### 监督投诉方式
        
        pat = '<h2.*?>咨询监督</h2>(.*?)id="set-basis"'
        s1 = re.search(pat,text,re.S)
        if s1 == None:
            self.text = None
        else:
            s1 = s1.groups()[0]
            self.text = s1
        
        pat1 = r'<div class="gd-col gd-col-6">(.*?)</div>'
        self.ab = re.findall(pat1,self.text,re.S)
        
    def get_consult(self):
        ''' 获得咨询方式 '''
        pat1 = '<p class="inquire-title">(.*?):</p>'
        k = re.findall(pat1,self.ab[0],re.S)
#         print(k)
        pat2 = '<p class="inquire-content">(.*?)</p>'
        v = re.findall(pat2,self.ab[0],re.S)
#         print(v)
        for a,b in zip(k,v):
            self.consult[a] = b
            
    def get_complaint(self):
        ''' 监督投诉方式 '''
        pat1 = '<p class="inquire-title">(.*?):</p>'
        k = re.findall(pat1,self.ab[1],re.S)
        if len(k) == 0:
            pat = '</h3>(.*)'
            ts = re.search(pat,self.ab[1],re.S)
            ts = ts.groups()[0].replace('\n','').replace('\t','')
            ts = ts.split('：')
            if len(ts) >= 2:
                self.complaint[ts[0]] = ts[1]
            else:
                self.complaint['投诉电话'] = ts[0]
#             print(ts)
        else:
            pat2 = '<p class="inquire-content">(.*?)</p>'
            v = re.findall(pat2,self.ab[1],re.S)
            for a,b in zip(k,v):
                self.complaint[a] = b
                
    def handle_sup(self):
        self.get_consult()
        self.get_complaint()
        
        tc1 = ['咨询方式','监督投诉方式']
        tc2 = [self.consult,self.complaint]
        
        self.fill_content(tc1,tc2)
#         self.content['咨询方式'] = [self.consult]
#         self.content['监督投诉方式'] = [self.complaint]

# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001033001'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(0.5)
# text = browser.page_source

# bs = BasicSup(text)
# bs.handle_sup()
# print(bs.complaint)
# print(bs.consult)

# '''
# OUTPUT
# # {'监督投诉电话': '0755-12345'}
# #{'咨询电话': '电话咨询：深圳市海洋综合执法支队大鹏大队：0755- 84400876；
# #             深圳市海洋综合执法支队盐田大队：0755-82630017；
# #             深圳市海洋综合执法支队宝安大队：0755- 27780739；深圳市海洋综合执法支队蛇口大队：0755-26693456。'}
# '''

In [39]:
## matters-scrollspy
class BasicRem(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        self.review = {}  ### 行政复议
        self.sue = {}     ### 行政诉讼
        
        pat = '<h2.*?>法律救济</h2>(.*?)id="matters-scrollspy"'
        s1 = re.search(pat,text,re.S)
        if s1 == None:
            self.text = None
        else:
            s1 = s1.groups()[0]
            self.text = s1
        
        pat1 = r'<div class="gd-col gd-col-6">(.*?)</div>'
        self.ab = re.findall(pat1,self.text,re.S)
    
    def get_inp(self,text):
        ''' 获得<p>...</p>中的内容 '''
        pat = '<p>(.*?)</p>'
        tb = re.search(pat,text,re.S)
        if tb == None:
            return ""
        else:
            tb = tb.groups()[0]
            return replace_text(tb)
        
    def get_review(self):
        ''' 获得行政复议的内容'''
        a = self.get_inp(self.ab[0])
        a = a.split('<br>')
        a1 = [ta.split('：') for ta in a]
        for ta1 in a1:
            self.review[ta1[0]] = ta1[1]
            
    def get_sue(self):
        b = self.get_inp(self.ab[1])
        b = b.split('<br>')
        b1 = [tb.split('：') for tb in b]
        for tb1 in b1:
            self.sue[tb1[0]] = tb1[1]
            
    def handle_rem(self):
        self.get_review()
        self.get_sue()
        
        tc1 = ['行政复议','行政诉讼']
        tc2 = [self.review,self.sue]
        self.fill_content(tc1,tc2)
        
#         self.content['行政复议'] = [self.review]
#         self.content['行政诉讼'] = [self.sue]

# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001033001'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(0.5)
# text = browser.page_source

# br = BasicRem()
# br.handle_rem()
# print(br.review)
# print(br.sue)

# '''
# OUTPUT
# {'部门': '深圳市规划和自然资源局', '地址': '深圳市福田区红荔西路8009号规划大厦', '电话': '0755-83788585', '网址': '无'}
# {'部门': '广州海事法院', '地址': '广州市海珠区南洲路863号', '电话': '020-34063863', '网址': '无'}

# '''

In [40]:
class BasicDeal(BasicPage):
    ''' 处理办理流程 '''
    def __init__(self,text):
        super().__init__()
        self.online = {}
        self.outline = {}
        soup = BeautifulSoup(text,'lxml')#选择lxml作为解析器
        tb1 = soup.find_all(id='tab1')
        if len(tb1) == 0:
            tb1 = soup.find_all(id='li_tab1')
        self.text1 = str(tb1[0])
        
        tb2 = soup.find_all(id='tab2')
        if len(tb2) == 0:
            tb2 = soup.find_all(id='li_tab2')
        self.text2 = str(tb2[0])
        

        
    def get_filepath(self,text):
#         print(text)
        pat = '"FILEPATH":"(.*?)"'
        url = re.findall(pat,text,re.S)
        if len(url) == 0:
            pat = 'FILEPATH&quot;:&quot;(.*?)&quot'
            url = re.findall(pat,text,re.S)
    #     print(url)
        return url[0]
    
    def get_inp(self,text):
        ''' 获得<p>...</p>中的内容 '''
        pat = '<p.*?>(.*?)</p>'
        tb = re.search(pat,text,re.S)
        if tb == None:
            return ""
        else:
            tb = tb.groups()[0]
            return replace_text(tb)
    
    def fillc(self,order):
        '''
        BeautifulSoup后的防止报错操作
        '''
        if len(order) >= 0:
            order = str(order[0])
            order = replace_brac(order)
        else:
            order = ''
        return order
    
    def get_pro_content(self,text):
        res = []
        soup = BeautifulSoup(text,'lxml')
        items = soup.find_all(class_ = 'matters-process-item')
        for item in items:
            rd = {}
            
            order = item.find_all(class_="matters-process-item-order")
            order = self.fillc(order)
            rd['序号'] = order
            
            header = item.find_all(class_ = 'matters-process-item-header')
            if len(header) == 0:
                header = ''
            else:
                header = str(header[0])
            
            ## 收件等跟在序号后面的文字
            act = S_search('<p>(.*?)</p>',header)
            rd['环节'] = act
            
            tp = re.findall('<li>(.*?)</li>',header,re.S) # time,person
#             print(tp)
            ### 直接认为有该信息
            time = tp[0]
            time = time.split('：')
            rd[time[0]] = time[1]
            
            person = tp[1]
            person = person.split('：')
            rd[person[0]] = person[1]
            

            st = item.find_all(class_="matters-process-item-text")
            if len(st) >= 0:
                st = str(st[0])
                st = replace_brac(st)
            else:
                st = ''
            tips = item.find_all(class_="matters-process-item-tips")
#             st = item.find_all(class_="matters-process-item-text")
            if len(tips) >= 0:
                tips = str(tips[0])
                tips = replace_brac(tips)
            else:
                tips= ''
            rd['标准'] = st+'\n'+tips
            res.append(rd)
        return res
#             print(type(item))
        
#         print(items[0])
        
    def get_step(self):
        '''获得步骤信息（线上线下相同）'''
        
        title = S_search('matters-process-title.*?>(.*?)<',self.text1)
        if title != '':
#             print(title)
            res = self.get_pro_content(self.text1)
#             print(res)
            self.content[title] = res
        else:  ## 获得一行字
            pstr = self.get_inp(self.text1)
            if pstr == "":
                return
            else:
    #             print(pstr)
                if '<br>' in pstr:
                    p = pstr.split('<br>',1)
                elif '<br/>' in pstr:
                    p = pstr.split('<br/>',1)
                else:
                    p =  ['办理流程',pstr]

                self.online[p[0]] = replace_brac(p[1])
                return
        
    def get_online(self):
        ''' 得到线上信息'''
             
        if re.search('FILEPATH',self.text1) == None:
            url = 'None'
        else:
#             print()
            url = self.get_filepath(self.text1)
            url = 'http:'+url
        self.online['线上办理流程说明'] = url   ### 流程图url
        
        
        
 
    def get_outline(self):
        ''' 得到线下的流程图url '''
        
        if re.search('FILEPATH',self.text2) == None:
            url = 'None'
        else:
            url = self.get_filepath(self.text2)
            url = 'http:'+url
        self.outline['线下办理流程说明'] = url
        
        
        
    def handle_deal(self):
        self.get_step()
        self.get_online()
        self.get_outline()
        
        tc1 = ['网上办理流程','线下办理流程']
        tc2 = [self.online,self.outline]
        self.fill_content(tc1,tc2)
        
#         self.content['网上办理流程'] = [self.online]
#         self.content['线下办理流程'] = [self.outline]

# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001033001'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(0.5)
# text = browser.page_source
# bd = BasicDeal(text)
# bd.handle_deal()
# print(bd.online)
# print(bd.outline)

# '''
# OUTPUT
# {'线上办理流程说明': 'None'}
# {'线下办理流程说明': 
#          'http://static.gdzwfw.gov.cn/obs/obs-sxml/txyCos6EEA7222D73DD7C7CA47D2AC14FA4DB54BD7EBBEF2BEE759BD850F0411EA6BD7619E29625B3D0D6C'}
# '''

In [41]:
class BasicSp(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        self.setting = []
        self.implement = {}
        soup = BeautifulSoup(text,'lxml')#选择lxml作为解析器
        
        tb3 = soup.find_all(id='tab3')
        if len(tb3) == 0:
            tb3 = soup.find_all(id='li_tab3')
        self.text1 = str(tb3[0])
        
        tb4 = soup.find_all(id='tab4')
        if len(tb4) == 0:
            tb4 = soup.find_all(id='li_tab4')
        self.text2 = str(tb4[0])
        
#         self.text1 = str(soup.find_all(id='tab3')[0])
#         self.text2 = str(soup.find_all(id='tab4')[0])
    
    def get_tr(self,text):
        pat = '<tr.*?>.*?<th width.*?>(.*?)</th>.*?<td>(.*?)</td>.*?</tr>'
        tr = re.findall(pat,text,re.S)
        return tr
    
    def get_setting(self):
        trs = self.get_tr(self.text1)
#         print(trs)
        trs_len = len(trs)
        i = 0
        while(i<trs_len):
            tr_dict = {}
            for j in range(6):
                a = trs[i][0]
                b = trs[i][1]
#                 print(a,b)
                b = replace_brac(b)
                i += 1
                tr_dict[a] = b
            
            self.setting.append(tr_dict)
    
    def get_implement(self):
        trs = self.get_tr(self.text2)
        if len(trs) == 0:
            return
        for a,b in trs:
            b = replace_brac(b)
            self.implement[a] = b
            
    def handle_sp(self):
        self.get_setting()
        self.get_implement()
        
        tc1 = ['设定依据','实施依据']
        tc2 = [self.setting,self.implement]
        
        self.fill_content(tc1,tc2)
        
#         self.content['设定依据'] = self.setting
#         self.content['实施依据'] = [self.implement]

        
# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001033001'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(0.5)
# text = browser.page_source
# bs = BasicSp(text)
# bs.handle_sp()
# print(bs.setting)
# print(bs.implement)

# '''
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '无有效的渔业船舶船名、船号、船舶登记证书（或船舶国籍证书）、检验证书的船舶，禁止其离港，并对船舶所有者或者经营者处船价２倍以下的罚款。有下列行为之一的，从重处罚：'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '无有效的渔业船舶船名、船号、船舶登记证书（或船舶国籍证书）、检验证书的船舶，禁止其离港，并对船舶所有者或者经营者处船价２倍以下的罚款。有下列行为之一的，从重处罚：'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第一项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '无有效的渔业船舶登记证书（或渔业船舶国籍证书）和检验证书，擅自刷写船名、船号、船籍港的；'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第一项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '无有效的渔业船舶登记证书（或渔业船舶国籍证书）和检验证书，擅自刷写船名、船号、船籍港的；'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第二项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '伪造渔业船舶登记证书（或国籍证书）、船舶所有权证书或船舶检验证书的；'},
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第二项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '伪造渔业船舶登记证书（或国籍证书）、船舶所有权证书或船舶检验证书的；'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第三项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '伪造事实骗取渔业船舶登记证书或渔业船舶国籍证书的；'},
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第三项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '伪造事实骗取渔业船舶登记证书或渔业船舶国籍证书的；'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第四项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '冒用他船舶名、船号或船舶证书的。'}, 
# {'法律法规名称': '中华人民共和国渔业港航监督行政处罚规定', '依据文号': '中华人民共和国农业部令第34号', '条款号': '第三章第一十六条第一款第四项', '颁布机关': ' 农业部(已撤销)', '实施日期': '2000-06-12', '条款内容': '冒用他船舶名、船号或船舶证书的。'}]
# *******************************
# {}
# '''

In [42]:
class RightDuty(BasicPage):
    
    def __init__(self,text):
        super().__init__()
        pat = '<h3>申请人依法享有以下权利</h3>(.*?)</p>'
        self.text1 = S_search(pat,text)
        
        pat = '<h3>申请人依法履行以下义务</h3>(.*?)</p>'
        self.text2 = S_search(pat,text)
        
        self.right = ''
        self.duty = ''
        
    def handle_rt(self):
        if self.text1 == "":
            return
        self.right = replace_text(replace_brac(self.text1))
        self.duty = replace_text(replace_brac(self.text2))
        
        tc1 = ['申请人依法享有以下权利','申请人依法履行以下义务']
        tc2 = [self.right,self.duty]
        
        self.fill_content(tc1,tc2)

# rt = RightDuty(text)
# rt.handle_rt()

# rt.content

In [43]:
class BasicQas(BasicPage):
    '''
    处理问答页面的信息，目前默认只有一页的问答信息
    '''
    
    def __init__(self,browser):
        super().__init__()
        self.browser = browser
        text1 = browser.page_source
        if '"matters-aside-box-title">问题' in text1:
            pass
        else:
            time.sleep(1)
        #### 与py文件不一致了 py文件这里有try:except
        if 'aside-faq">无<'  not in text1 and '"matters-aside-box-title">问题' in text1:
            try:
                xpath = '//*[@id="matters-detail"]/div[1]/div[2]/div[1]/div/div/div[2]/div[4]/div[2]/div[1]/div[2]/ul/a'
                self.browser.find_element_by_xpath(xpath).click()
                time.sleep(1)
                n = self.browser.window_handles
        #         print(n)
                self.browser.switch_to_window(n[-1])
                self.qatext = self.browser.page_source

                soup = BeautifulSoup(self.qatext,'lxml')
                self.qas = soup.find_all(class_ = 'matters-faq')
            except:
                self.qatext = ''
        else:
            self.qatext = ''
        
        self.res = []
        
#     def __del__(self):
#         n = self.browser.window_handles
#         browser = self.browser
#         self.browser.switch_to_window(n[0])
#         browser.close()
        
        
    def get_qas(self):
#         res = []
        qas = self.qas[0]
        items = qas.find_all(class_="matters-faq-item")
        for item in items:
            tdict = {}
            item = str(item)
            q = S_search('<div class="matters-faq-question">(.*?)</div>',item)
            tdict['问题'] = q
            ans = S_search('<div class="matters-faq-answer">(.*?)</div>',item)
            tdict['答案'] = ans
            self.res.append(tdict)
            
    def handle_qas(self):
#         print(self.qas)
        if self.qatext == '' or len(self.qas) == 0:
            return
        self.get_qas()
        self.content['问答'] = self.res

# browser.switch_to_window('18')
# qas = BasicQas(browser)

# qas.handle_qas()
# qas.content

In [44]:
# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300695587927L3440119022003'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)

# qas = BasicQas(browser)

# qas.qatext

# qas.handle_qas()

# qas.content

#### 利用上述部分构建新类处理页面

In [45]:
class Guide(BasicPage):
    
    def __init__(self,browser,text,url,idx,pid):
        super().__init__()
        self.content['url'] = url
        self.content['last_level_id'] = idx
        self.content['pid'] = pid
        self.infos = BasicInfo(text)  ### 基本信息
        self.accept = BasicAccept(text) ### 受理标准*
        self.procedure = BasicDeal(text) ### 办理流程
        self.meter = Materials(text) ### 申请材料*
        self.supervise = BasicSup(text) ## 咨询监督
        self.window = BasicWin(text) ## 窗口办理*
        self.fee = BasicFee(text)   ## 收费项目*
        self.agent = BasicAgent(text) ## 中介服务*
        self.sp = BasicSp(text)     ## 法律依据
        self.rt = RightDuty(text)  ## 权利和义务
        self.remedy = BasicRem(text) ## 法律救济
        
        self.qas = BasicQas(browser)  ## 必须最后处理
    
    def get_infos(self):
        ''' 利用上面的类生成基本信息'''
        self.infos.handle_infos()
        
    def get_supervise(self):
        ''' 处理咨询监督部分'''
#         self.supervise = BasicSup(text)
        self.supervise.handle_sup()
    
    def get_procedure(self):
        '''处理办事流程部分'''
        self.procedure.handle_deal()
        
    def get_remedy(self):
        self.remedy.handle_rem()
        
    def get_sp(self):
        self.sp.handle_sp()
    
    def get_accept(self):
        self.accept.handle_acc()
        
    def get_meter(self):
        self.meter.handle_appli()
        
    def get_window(self):
        self.window.handle_win()
        
    def get_fee(self):
        self.fee.handle_fee()
        
    def get_agent(self):
        self.agent.handle_agent()
        
    def get_rt(self):
        self.rt.handle_rt()
        
    def get_qas(self):
        self.qas.handle_qas()
        
    def handle_guide(self):
        self.get_infos()
        self.get_supervise()
        self.get_procedure()
        self.get_procedure()
        self.get_remedy()
        self.get_sp()
        self.get_accept()
        self.get_meter()
        self.get_window()
        self.get_fee()
        self.get_agent()
        self.get_rt()
        
        self.get_qas() ## 为了最后处理该信息
        
        tc1 = ['基本信息',  
               '受理标准',  '办理流程', 
               '申请材料','咨询监督', 
               '窗口办理','收费项目信息','中介服务',
               '法律依据','权利与义务','法律救济','问答'
              ]
        tc2 = [self.infos.content,  
               self.accept.content,  self.procedure.content,
               self.meter.content,self.supervise.content,
               self.window.content,self.fee.content,self.agent.content,
               self.sp.content,self.rt.content,self.remedy.content,self.qas.content]
        self.fill_content(tc1,tc2)
        
        
#         self.content['基本信息'] = self.infos.content
#         self.content['办理流程'] = self.procedure.content
#         self.content['咨询监督'] = self.supervise.content
#         self.content['法律依据'] = self.sp.content
#         self.content['法律救济'] = self.remedy.content
        


In [46]:
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/TE44030000754666303440737003000'
# turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3440101003001'
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440100MB2C9235X13440217281000'
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(2)
# text = browser.page_source

# guide = Guide(browser,text,'123456',0,0)
# guide.handle_guide()
# del guide
#     guide.content

In [47]:
# # 5项 10  11 都可以解析
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/TE44030000754666303440737003000'
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440100MB2C9235X13440217281000' 5
# # turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300695587927L3440119001000'  11
# browser = webdriver.Firefox(log_path = "webdriver.log")
# browser.get(turl)
# time.sleep(2)
# text = browser.page_source

# guide = Guide(text,'123456',0)
# guide.handle_guide()

# guide.content

In [48]:
# '''
# http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001028000  窗口信息 申请材料 以及 办理流程

# '''

In [49]:
if __name__ == '__main__':
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/91440300192408392D3442199053001'
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/TE44030000754666303440737003000'
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440100MB2C9235X13440217281000'
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440100MB2C9235X13440217281000'
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300MB2C94128A3440217281000'
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300693966093K3442001033001'
    turl = 'http://www.gdzwfw.gov.cn/portal/guide/91440300192408392D3442199053001'
#     turl = 'http://www.gdzwfw.gov.cn/portal/guide/11440300695587927L3440119022003'
    browser = webdriver.Firefox(log_path = "webdriver.log")
    browser.get(turl)
    time.sleep(2)
    text = browser.page_source

#     browser.close()

    guide = Guide(browser,text,turl,0,0)
    guide.handle_guide()
    with open('14.json','w+',encoding='utf-8') as fout:
        guide.disp(fout)
#         fout.write('********')

C:\Users\as\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: use service_log_path instead of log_path
  # Remove the CWD from sys.path while we load stuff.
